In [13]:
# riss.kr 에서 특정 키워드로 논문 / 학술 자료 검색하기

#Step 1. 필요한 모듈을 로딩합니다
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import time          

#Step 2. 사용자에게 검색 관련 정보들을 입력 받습니다.
print("=" *100)
print(" 이 크롤러는 RISS 사이트의 논문 및 학술자료 수집용 웹크롤러입니다.")
print("=" *100)
query_txt = input('1.수집할 자료의 키워드는 무엇입니까? : ')

#Step 3. 수집된 데이터를 저장할 파일 이름 입력받기 
ft_name = input('2.결과를 저장할 txt형식의 파일명을 쓰세요(예: c:\\py_temp\\riss.txt): ')
fc_name = input('3.결과를 저장할 csv형식의 파일명을 쓰세요(예: c:\\py_temp\\riss.csv): ')
fx_name = input('4.결과를 저장할 xls형식의 파일명을 쓰세요(예: c:\\py_temp\\riss.xls): ')

#Step 4. 크롬 드라이버 설정 및 웹 페이지 열기
s = Service("c:/py_temp/chromedriver.exe")
driver = webdriver.Chrome(service=s)

url = 'https://www.riss.kr/'
driver.get(url)
time.sleep(5)
driver.maximize_window()

#Step 5. 자동으로 검색어 입력 후 조회하기
element = driver.find_element(By.ID,'query')
driver.find_element(By.ID,'query').click( )
element.send_keys(query_txt)
element.send_keys("\n")

#Step 6.학위 논문 선택하기
driver.find_element(By.LINK_TEXT,'학위논문').click()
time.sleep(2)

#Step 7.Beautiful Soup 로 본문 내용만 추출하기
from bs4 import BeautifulSoup
html_1 = driver.page_source
soup_1 = BeautifulSoup(html_1, 'html.parser')

content_1 = soup_1.find('div','srchResultListW').find_all('li')
for i in content_1 :
    print(i.get_text().replace("\n",""))

#Step 8. 총 검색 건수를 보여주고 수집할 건수 입력받기
import math
total_cnt = soup_1.find('div','searchBox pd').find('span','num').get_text()
print('검색하신 키워드 %s (으)로 총 %s 건의 학위논문이 검색되었습니다' %(query_txt,total_cnt))
collect_cnt = int(input('이 중에서 몇 건을 수집하시겠습니까?: '))
collect_page_cnt = math.ceil(collect_cnt / 10)
print('%s 건의 데이터를 수집하기 위해 %s 페이지의 게시물을 조회합니다.' %(collect_cnt,collect_page_cnt))
print('=' *80)

#Step 9. 각 항목별로 데이터를 추출하여 리스트에 저장하기
no2 = [ ]        #번호 저장
title2 = [ ]     #논문제목 저장
writer2 = [ ]    #논문저자 저장
org2 = [ ]       #소속기관 저장
no = 1

for a in range(1, collect_page_cnt + 1) :
    
    html_2 = driver.page_source
    soup_2 = BeautifulSoup(html_2, 'html.parser')

    content_2 = soup_2.find('div','srchResultListW').find_all('li')
    
    for b in content_2 :    
        #1. 논문제목 있을 경우만
        try :
            title = b.find('div','cont').find('p','title').get_text()
        except :
            continue
        else :
            f = open(ft_name, 'a' , encoding="UTF-8")
            print('1.번호:',no)
            no2.append(no)
            f.write('\n'+'1.번호:' + str(no))

            print('2.논문제목:',title)
            title2.append(title)
            f.write('\n' + '2.논문제목:' + title)
            
            writer = b.find('span','writer').get_text()
            print('3.저자:',writer)
            writer2.append(writer)
            f.write('\n' + '3.저자:' + writer)

            org = b.find('span','assigned').get_text()
            print('4.소속기관:' , org)
            org2.append(org)
            f.write('\n' + '4.소속기관:' + org + '\n')
            
            f.close( )
            
            no += 1
            print("\n")
            
            if no > collect_cnt :
                break

            time.sleep(1)        # 페이지 변경 전 1초 대기 

    a += 1 
    b = str(a)

    try :
        driver.find_element(By.LINK_TEXT ,'%s' %b).click() 
    except :
        driver.find_element(By.LINK_TEXT('다음 페이지로')).click()
        
print("요청하신 작업이 모두 완료되었습니다")

# Step 10. 수집된 데이터를 xls와 csv 형태로 저장하기
import pandas as pd 

df = pd.DataFrame()
df['번호']=no2
df['제목']=pd.Series(title2)
df['저자']=pd.Series(writer2)
df['소속(발행)기관']=pd.Series(org2)

# xls 형태로 저장하기
df.to_excel(fx_name,index=False, encoding="utf-8" , engine='openpyxl')

# csv 형태로 저장하기
df.to_csv(fc_name,index=False, encoding="utf-8-sig")

print('요청하신 데이터 수집 작업이 정상적으로 완료되었습니다')

 이 크롤러는 RISS 사이트의 논문 및 학술자료 수집용 웹크롤러입니다.
1.수집할 자료의 키워드는 무엇입니까? : 해양자원
2.결과를 저장할 txt형식의 파일명을 쓰세요(예: c:\py_temp\riss.txt): c:\py_temp\riss.txt
3.결과를 저장할 csv형식의 파일명을 쓰세요(예: c:\py_temp\riss.csv): c:\py_temp\riss.csv
4.결과를 저장할 xls형식의 파일명을 쓰세요(예: c:\py_temp\riss.xls): c:\py_temp\riss.xls
1지속가능한 해양관광 발전방안에 대한 연구 : 관광자원개발을 중심으로김선종연세대학교 행정대학원2008국내석사RANK : 13880191원문보기음성듣기삼면이 바다로 둘러싸여 천혜의 해양국가인 우리나라는 세계적 수준의 자연관광자원을 보유하고 있으며 연안을 중심으로 관광활동에 좋은 지형적 지리적 조건을 갖추고 있다. 특히, 우리나라 해안은 지형 및 지세, 기상 등 자원특성이 서로 달라 각 해안별로 독특한 자연조건을 갖고 있어 해양관광발전 잠재력이 매우 크다. 수질이 깨끗하고 백사장이 잘 발달된 동해안은 해수욕에 유리하고, 수심이 낮고 갯벌이 발달된 서해안은 생태관광의 보고이며, 기후가 온난하고 호수와 같은 잔잔한 내해를 이루고 있는 남해안은 스포츠형관광의 최적의 자연자원을 보유하고 있다.바다와 육지가 맞닿아 서로 밀접한 영향을 주고받으면서 특수한 자원?환경시스템을 이루는 연안(沿岸)은 해변?갯벌?만?삼각주 등 다양한 모습을 보이고 있으며, 그동안 수산물의 산란장과 서식지로서 항만?임해공단 등 경제활동공간으로서 도시가 발달하고 인구가 집중되어있는 중요한 활동공간이었으나, 이제는 레저스포츠 등 관광활동공간으로서 그 중요성이 더해가고 있다.우리나라는 우수한 자연관광자원을 보유하고 있으나 그동안 육상위주의 레저 및 관광정책추진으로 해양관광은 크게 발전하지 못하고 있다. 그러나 국민들의 소득수준 증가와 더불어 주2일 휴무제 정착에 따른 여가시간의 증대 등 라이프스타일 변화로 직

이 중에서 몇 건을 수집하시겠습니까?: 25
25 건의 데이터를 수집하기 위해 3 페이지의 게시물을 조회합니다.
1.번호: 1
2.논문제목: 지속가능한 해양관광 발전방안에 대한 연구 : 관광자원개발을 중심으로
3.저자: 김선종
4.소속기관: 연세대학교 행정대학원


1.번호: 2
2.논문제목: 島嶼觀光資源開發이 海洋觀光産業에 미치는 經濟的 波及效果
3.저자: 김두성
4.소속기관: 부산대학교


1.번호: 3
2.논문제목: 여수지역 해양생물자원을 활용한 관광지 활성화에 관한 연구
3.저자: 권진구
4.소속기관: 전남대학교 대학원


1.번호: 4
2.논문제목: 해양관광유형과 관광자원 가치가 주민참여의도에 미치는 영향 : 당진시를 중심으로
3.저자: 이길호
4.소속기관: 경기대학교 관광전문대학원


1.번호: 5
2.논문제목: 국가관할권 이원지역 해양유전자원의 접근 및 이익공유에 관한 연구
3.저자: 조은영
4.소속기관: 중앙대학교 대학원


1.번호: 6
2.논문제목: 해양경계획정 이전 권원중첩수역에서의 자원개발과 환경보호
3.저자: 이영주
4.소속기관: 연세대학교 대학원


1.번호: 7
2.논문제목: 海洋觀光資源開發에 관한 硏究 : 邊山半島를 中心으로
3.저자: 박명엽
4.소속기관: 경기대학교 대학원


1.번호: 8
2.논문제목: 해양관광 자원 개발에 대한 연구 : 중국 칭다오와 부산을 중심으로
3.저자: 장잉잉
4.소속기관: 경기대학교 대학원


1.번호: 9
2.논문제목: GIS기반 해양광물자원정보시스템 개발에 관한 연구
3.저자: 김동일
4.소속기관: 인하대학교 대학원


1.번호: 10
2.논문제목: 국가관할권이원지역 해양생물자원의 이용과 규율 : UN해양법협약 원칙의 확장적 적용가능성을 중심으로
3.저자: 서진웅
4.소속기관: 성균관대학교


1.번호: 11
2.논문제목: 韓·中의 海洋觀光資源 開發에 관한 연구 : 釜山 靑島를 中心으로
3.저자: 왕염
4.소속기관: 동신대학교 대학원


1.번호: 12
2.논문제목: 해양관광자원의 경제적 가치평가 